## Exercice 1

In [1]:
import json 

In [19]:
#Reading training_set file 
with open('training_set.json', encoding="utf8") as json_file:
    training_data = json.load(json_file)
    
#Reading testing_set file 
with open('testing_set.json', encoding="utf8") as json_file:
    testing_data = json.load(json_file)

In [61]:
#Counting the number of data (train) for each intent
train_data = {'find-train': 0, 'irrelevant': 0, 'find-flight': 0, 'find-restaurant': 0,
           'purchase': 0, 'find-around-me' : 0, 'provide-showtimes' : 0, 'find-hotel' : 0}
for i in range(len(training_data)):
    for intent, count in train_data.items():
        if training_data[i]['intent'] == intent :
            count = count + 1
            train_data[intent] = count

#Counting each intent iteration
train_data

{'find-train': 143,
 'irrelevant': 3852,
 'find-flight': 142,
 'find-restaurant': 469,
 'purchase': 613,
 'find-around-me': 383,
 'provide-showtimes': 117,
 'find-hotel': 316}

In [63]:
#Counting the number of data (test) for each intent
test_data = {'find-train': 0, 'irrelevant': 0, 'find-flight': 0, 'find-restaurant': 0,
           'purchase': 0, 'find-around-me' : 0, 'provide-showtimes' : 0, 'find-hotel' : 0}
for i in range(len(testing_data)):
    for intent, count in test_data.items():
        if testing_data[i]['intent'] == intent :
            count = count + 1
            test_data[intent] = count

#Counting each intent iteration
test_data

{'find-train': 21,
 'irrelevant': 677,
 'find-flight': 24,
 'find-restaurant': 93,
 'purchase': 114,
 'find-around-me': 67,
 'provide-showtimes': 14,
 'find-hotel': 55}

## Exercice 2

## Exercice 3

## Exercice 4

## Exercice 5